<a href="https://colab.research.google.com/github/shawn-kg/DataMining/blob/main/assign5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Preprocessing

In [78]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=True, with_std=True)
data = pd.read_csv('./OnlineNewsPopularity.csv')

data = data.drop(data.columns[[0,1,4,5,6,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38]], axis=1)
# print(data.shape)
data = data.to_numpy()

np.random.seed(42)
np.random.shuffle(data)
target = data[:,-1]
data = np.delete(data,-1,1)
# print(data)
train = data[:1000]
# print(train)
valid = data[1000:2000]
# print(valid.shape)
test = data[2000:3000]
# print(test.shape)
train_means = np.mean(train, axis=0)
# print(train_means)
train_std = np.std(train, axis=0)
# scaler.fit(train)
# train = scaler.transform(train)
# print(train_std)
for i in range(1000):
  for j in range(29):
    train[i][j] = (train[i][j] - train_means[j])/train_std[j]
for i in range(1000):
  for j in range(29):
    valid[i][j] = (valid[i][j] - train_means[j])/train_std[j]
for i in range(1000):
  for j in range(29):
    test[i][j] = (test[i][j] - train_means[j])/train_std[j]
# train = np.insert(train,0,np.ones((1000)),axis=1)
# valid = np.insert(valid,0,np.ones((1000)),axis=1)
# test = np.insert(test,0,np.ones((1000)),axis=1)
print("Train")
print(train)
print("Valid")
print(valid)
print("Test")
print(test)

new_target = np.zeros((len(target)))
for i in range(len(target)):
  if target[i]>6000:
    new_target[i] = 6000
  else:
    new_target[i] = target[i]
target_class = np.zeros((len(target)))
for i in range(len(target)):
  if new_target[i]>=2000:
    target_class[i] = 1
  else:
    target_class[i] = -1
print(target_class)


Train
[[ 0.7240996   1.06244462 -0.05217198 ... -0.23790414  0.82455503
  -0.67891922]
 [-0.69942517 -0.04158213 -0.14059905 ... -0.23790414  0.82455503
  -0.67891922]
 [-0.69942517 -0.62594004  0.12468218 ... -0.23790414  0.82455503
  -0.67891922]
 ...
 [ 0.24959134  1.78911848  1.09738003 ...  3.56205275 -0.77290887
   3.78934244]
 [ 0.7240996   0.12876575  0.21310926 ... -0.23790414  0.82455503
  -0.67891922]
 [ 1.67311611  0.42633546 -0.05217198 ...  1.47207646 -1.57164083
   1.33179853]]
Valid
[[-0.69942517  0.23226825 -0.31745321 ... -0.23790414  0.82455503
  -0.67891922]
 [-0.22491691  1.05381941  0.8320988  ...  0.52208724 -0.24042091
   0.21473311]
 [ 1.19860785 -0.7790375  -0.31745321 ...  0.23709047 -0.50666489
  -0.12038651]
 ...
 [-0.22491691 -0.7725686   1.27423419 ... -0.23790414  0.82455503
  -0.67891922]
 [-0.22491691 -0.81785094 -0.67116152 ... -0.23790414  0.82455503
  -0.67891922]
 [-1.17393342  3.01389816  4.01547359 ...  1.6620743   0.82455503
   1.55521161]]
Test

# SVM

In [79]:
def SVM(K,target_class,epsilon,C):
  K_aug = K+1
  eta = np.zeros((1000))
  for k in range(K_aug.shape[0]):
    eta[k] = 1/K_aug[k][k]
  alpha = np.random.random_sample((1000))
  alpha_prev = np.zeros(1000)
  target_train = target_class[:1000]
  shuffled = np.array(range(1000))
  while(True):
    np.random.shuffle(shuffled)
    alpha_prev = np.copy(alpha)
    for k in shuffled:
      alpha[k] += eta[k]*(1 - target_train[k]* np.sum(alpha*target_train*K_aug[:,k]))
      if alpha[k]<0:
        alpha[k] = 0
      if alpha[k] > C:
        alpha[k] = C
    if (np.linalg.norm(alpha -alpha_prev)<=epsilon):
      break
  return alpha

def acc(target_class, lin_Kernel, alpha):
  lin_KernelSum = np.sum(lin_Kernel, axis=0)
  predicted = alpha*lin_KernelSum*target_class

  for i in range(1000):
    if predicted[i]>=0:
      predicted[i] = 1
    else:
      predicted[i] = -1

  correct = 0
  for i in range(1000):
    if predicted[i] == target_class[i]:
      correct+=1
  acc = correct/1000
  return acc


##Linear

In [80]:
K = np.dot(train, train.transpose())
epsilon = 0.1
C = .1
alpha = SVM(K,target_class, epsilon, C)
lin_Kernel = np.dot(train,valid.transpose())+1
valid_acc = acc(target_class[1000:2000], lin_Kernel, alpha)
print("Valid Accuracy: ", valid_acc)
lin_Kernel = np.dot(train,test.transpose())+1
test_acc = acc(target_class[2000:3000], lin_Kernel, alpha)
print("Test Accuracy: ", test_acc)
print("Epsilon: ", epsilon)
print("C: ", C)

Valid Accuracy:  0.936
Test Accuracy:  0.937
Epsilon:  0.1
C:  0.1


## Gaussian

In [81]:
def gaussian(matrix1, matrix2, spread):
  S = np.linalg.norm(matrix1, axis=1)**2
  R = np.linalg.norm(matrix2, axis=1)**2
  S = S.reshape(-1,1)
  R = R.reshape(1,-1)
  K = np.exp((2 * np.dot(matrix1,np.transpose(matrix2)) - S - R)/(2*spread))
  return K
epsilon = 0.1
C = 0.1
spread = 10000
GauK = gaussian(train,train,spread)
alpha = SVM(GauK, target_class, epsilon, C)
validGauK = gaussian(train,valid,10000)
valid_acc = acc(target_class[1000:2000], validGauK, alpha)
print("Valid Accuracy: ", valid_acc)
testGauK = gaussian(train,test,10000)
test_acc = acc(target_class[2000:3000], testGauK, alpha)
print("Test Accuracy: ", test_acc)
print("Epsilon: ", epsilon)
print("C: ", C)
print("Spread: ", spread)

Valid Accuracy:  0.942
Test Accuracy:  0.953
Epsilon:  0.1
C:  0.1
Spread:  10000


##Inhomogeneous Polynomial Kernel

In [82]:
def inPolyKernel(q,c,matrix1,matrix2):
  K_first = np.dot(matrix1,matrix2.transpose()) + c
  K = np.power(K_first, q)
  return K
q = 2
c = 100000
K = inPolyKernel(q,c,train,train)
epsilon = 0.1
C = 0.1
alpha = SVM(K, target_class, epsilon, C)
validK = inPolyKernel(q,c,train,valid)
valid_acc = acc(target_class[1000:2000], validK, alpha)
print("Valid Accuracy: ", valid_acc)
testK = inPolyKernel(q,c,train,test)
test_acc = acc(target_class[2000:3000], testK, alpha)
print("Test Accuracy: ", test_acc)
print("Epsilon: ", epsilon)
print("C: ", C)


Valid Accuracy:  0.818
Test Accuracy:  0.817
Epsilon:  0.1
C:  0.1


# Exam 1 Questions

##Question 5

In [85]:
import pandas as pd
import numpy as np
import math

data = np.loadtxt('./seeds_dataset.txt', dtype = float)
data = np.delete(data,7,1)
# print(data)
# print(data.shape)

K = np.dot(data,data.transpose())
K = np.power(K,2)

id = np.identity(210)
ones = np.ones((210,210))
ones = ones/210
center_helper = id - ones
center_helperi = np.dot(center_helper,K)
K_hat = np.dot(center_helperi,center_helper)
E, C = np.linalg.eigh(K_hat)

variances = E / 210
n = np.sum(variances)
C_unit = np.zeros((210,210))

for i in reversed(range(210)):
  C_unit[:,i] = (1/math.sqrt(abs(E[i])))*C[:,i]

phi = np.zeros((210,28))
count = 0
for i in range(7):
  for j in range(i,7):
    phi[:,count] = data[:,i]*data[:,j]
    if i!=j:
      phi[:,count]*=math.sqrt(2)
    count+=1
print(phi)

ui = np.zeros(28)
for i in (range(210)):
  ui+= C_unit[i][209]*phi[i]
print("Unit Vector:", ui)


[[232.8676     320.26054059  18.796963   ...   4.932841    16.39585464
   27.2484    ]
 [221.4144     306.60376306  18.54142592 ...   1.036324     7.13500158
   24.561936  ]
 [204.2041     284.74638535  18.28924618 ...   7.284601    18.4168436
   23.280625  ]
 ...
 [174.24       254.99967586  16.58244598 ...  69.139225    59.45444326
   25.563136  ]
 [140.1856     221.19205212  14.2678083  ...  12.945604    25.66558896
   25.441936  ]
 [151.29       232.04698974  15.10566761 ...  31.775769    40.36184033
   25.633969  ]]
Unit Vector: [-6.14781397e-01 -6.10913284e-01 -2.62906997e-02 -2.24972040e-01
 -1.46798782e-01 -7.15891808e-02 -2.22498680e-01 -2.63633966e-01
 -1.27865332e-02 -1.33307258e-01 -9.32728109e-02 -1.05458329e-02
 -1.33124652e-01 -1.55487710e-04 -4.30815137e-03 -3.43997711e-03
  1.79866636e-03 -4.39761953e-03 -3.34874838e-02 -3.36473658e-02
 -8.82792529e-04 -4.73779638e-02 -1.63031868e-02 -5.28717940e-03
 -3.35283471e-02  1.68706633e-02 -1.29344015e-03 -3.34370238e-02]


##Question 8

In [84]:
Ev1v2 = 0*(1/4) + 0*(1/4) + 0*(1/4) + 1*(1/4)
Ev1v1 = 0*(1/2) + 1*(1/2)
Ev2v2 = 0*(1/2) + 1*(1/2)
costheta = Ev1v2/(math.sqrt(Ev1v1)*math.sqrt(Ev2v2))
print(costheta)
angle = math.acos(costheta)
angle *= (180/math.pi)
print(angle,'degrees')

0.4999999999999999
60.00000000000001 degrees
